In [1]:
pip install tweet-preprocessor

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install textblob

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tweepy

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import tweepy
import re
import os             
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
import preprocessor as p
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 


#input tweeter credentials#
CONSUMER_KEY = '3bAsLVoFl8HmlJBwJAB3hATYO'
CONSUMER_SECRET = 'mOyV7MHc22fhxGJdUrvVjqHY4cUcsApPBHu0tyiAzenWxCM1wB'
OAUTH_TOKEN = '1310696980859899909-1E9G87iXHIUOyPQcHJqxF4nsHptPac'
OAUTH_TOKEN_SECRET = 'YPXkDZz2K0dny0NfdS0clfJ4uCCKQVZn0hGaTIR9HV5tN'


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


def removeNLTKStop(text):
    stop_words = set(stopwords.words('english')) 
    filtered_sentences = []
    w = []
    result_string=""
    
    text = re.sub(r'\.COM|[^a-zA-Z ]+|\s(?=&)|(?<!\w\w)\s+(?!\w\w)', '', text, 0, re.IGNORECASE)
    
    word_tokens = word_tokenize(text.lower()) 
    
    #Removed NLTK Stopwords 
    filtered_article = [w for w in word_tokens if not w in stop_words] 
    filtered_article = [] 
    
    ps = PorterStemmer() 
    stemmer = nltk.SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer() 
    #for s in word_tokens:
     #   s = re.sub(r'\.COM|[^a-zA-Z ]+|\s(?=&)|(?<!\w\w)\s+(?!\w\w)', '', s, 0, re.IGNORECASE)
     #   result_string = result_string+s
        
    for w in word_tokens: 
        #w = ps.stem(w)
        #w = stemmer.stem(w)
        
     #   result_string = result_string+s
        w = lemmatizer.lemmatize(w,pos="v") 
        if w not in stop_words: 
            filtered_article.append(w)
    return " ".join(filtered_article)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def translateTweet(tweet):
    #translate
    translator = Translator()
    tweet = translator.translate(tweet) 
    return tweet

def processTweet2(tweet):
    
  
    #Remove RT
    tweet = re.sub('RT','',tweet)
    #Convert to lower case
    tweet = tweet.lower()
    tweet = tweet.replace('-',' ', 1) 
    #tweet=re.sub(r'-(?:(?<!\b[0-9]{4}-)|(?![0-9]{2}(?:[0-9]{2})?\b))', ' ', tweet)

    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to ''
    tweet = re.sub('@[^\s]+','',tweet)
    
    tweet = re.sub('u.s.','us',tweet)
    tweet = re.sub('united states','us',tweet)


    tweet = re.sub('u.k.','uk',tweet)
    tweet = re.sub('united kingdom','uk',tweet)


    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #remove NLTK
    tweet = removeNLTKStop(tweet)
    
    
    #tweet = tweet.strip('\'"')
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    tweet.lstrip()


    #trim consecutive spaces
    tweet = re.sub(' +', ' ', tweet) 
    
    
    
    tweet = replaceTwoOrMore(tweet)

    
    # remove emoticons
    tweet = p.clean(tweet)
    return tweet    

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

[nltk_data] Downloading package stopwords to /home/tc2006/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tc2006/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def get_tweet_sentiment(tweet): 
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [19]:
from textblob import TextBlob 
import nltk
nltk.download('wordnet')

api = tweepy.API(auth)
tweet = api.get_status("1324827582861348866",tweet_mode="extended")
if 'retweeted_status' in dir(tweet):
    text=tweet.retweeted_status.full_text
                    #print("This is retweeted")
else:
    text=tweet.full_text
print("=====Original Tweet======= \n",text)
tweet=processTweet2(text)
print("=====Processed Tweet======= \n",tweet)

results =get_tweet_sentiment(tweet.rstrip())

print("=====Sentiment Polarity with Naive Bayes====== \n",results)


print("=====Sentiment Polarity with CoreNLP====== \n")
for s in result["sentences"]:
    print("{} (Sentiment Value) {} (Sentiment)".format(
   
        s["sentimentValue"], s["sentiment"]))

[nltk_data] Downloading package wordnet to /home/tc2006/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


=====Original Tweet======= 
 Also my dad was exposed to covid recently but has thankfully tested negative so maybe I'm catching a break this time
=====Processed Tweet======= 
 also dad expose covid recently thankfully test negative maybe im catch break time
=====Sentiment Polarity with Naiva Bayes====== 
 negative
=====Sentiment Polarity with CoreNLP====== 

1 (Sentiment Value) Negative (Sentiment)


In [1]:
pip install pycorenlp

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from pycorenlp import StanfordCoreNLP

In [3]:
pip install -U nltk[corenlp]

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: nltk[corenlp] in ./.local/lib/python3.5/site-packages (3.5)
Note: you may need to restart the kernel to use updated packages.


In [11]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [12]:
from nltk.parse import CoreNLPParser

In [13]:
result = CoreNLPParser(url='http://localhost:9000')

In [14]:
result = nlp.annotate(tweet,
                   properties={
                       'annotators': 'sentiment, ner, pos',
                       'outputFormat': 'json',
                       'timeout': 10000000000000000000000000000000000000000000000000000000000000,
                   })

In [21]:
for s in result["sentences"]:
    print("{}: '{}': {} (Sentiment Value) {} (Sentiment)".format(
        s["index"],
        " ".join([t["word"] for t in s["tokens"]]),
        s["sentimentValue"], s["sentiment"]))

0: 'also dad expose covid recently thankfully test negative maybe im catch break time': 1 (Sentiment Value) Negative (Sentiment)


In [22]:
result

{'sentences': [{'basicDependencies': [{'dep': 'ROOT',
     'dependent': 3,
     'dependentGloss': 'expose',
     'governor': 0,
     'governorGloss': 'ROOT'},
    {'dep': 'advmod',
     'dependent': 1,
     'dependentGloss': 'also',
     'governor': 3,
     'governorGloss': 'expose'},
    {'dep': 'dep',
     'dependent': 2,
     'dependentGloss': 'dad',
     'governor': 3,
     'governorGloss': 'expose'},
    {'dep': 'dobj',
     'dependent': 4,
     'dependentGloss': 'covid',
     'governor': 3,
     'governorGloss': 'expose'},
    {'dep': 'advmod',
     'dependent': 5,
     'dependentGloss': 'recently',
     'governor': 3,
     'governorGloss': 'expose'},
    {'dep': 'advmod',
     'dependent': 6,
     'dependentGloss': 'thankfully',
     'governor': 7,
     'governorGloss': 'test'},
    {'dep': 'nsubj',
     'dependent': 7,
     'dependentGloss': 'test',
     'governor': 10,
     'governorGloss': 'im'},
    {'dep': 'amod',
     'dependent': 8,
     'dependentGloss': 'negative',
    

In [ ]:
for s in result["sentences"]:
    print("{}: '{}': {} (Sentiment Value) {} (Sentiment)".format(
    s["index"],
    " ".join([t["word"] for t in s["tokens"]]),
    s["sentimentValue"], s["sentiment"]))
    if(s["sentimentValue"] == "0" or s["sentimentValue"] == "1" ):
        ntweet.append(text)
    if(s["sentimentValue"] == "3" or s["sentimentValue"] == "4" ):
        ptweet.append(text)
    if (s["sentimentValue"] == "2" ):
            neutweet.append(text)
    